In [3]:
import pyrogram
import pickle

pkl = pickle.load(open('data/messages-2.pkl', 'rb'))
dat = pkl["history"]
msgs = pkl["msgs"]

In [6]:
print(len(dat), len(msgs))

532 575


In [14]:
msg_tops = {}     # top levels cache by reply_to ids
tops = {}         # top levels data - counts, total_size, list of messages
msgs_quotes = {}  # quote counts for messages

for i, d in enumerate(reversed(dat)):
    if d.reply_to_message_id is None:
        print("new top", d)
        continue
    # get top from cache
    top = msg_tops.get(d.reply_to_message_id)
    if not top:
        # find real top
        top = d
        while top.reply_to_message_id:
            top = msgs.get(top.reply_to_message_id)
            if not top:
                print("WARNING: unknown top", d.reply_to_message_id)
                break
        if not top:
            print("WARNING: cant find top for", d)
            continue
        # fill cache
        msg_tops[d.reply_to_message_id] = top
        if top.id not in tops:
            tops[top.id] = [0, 0, []]
            print(f"New top: {top.id}")
    
    if d.reply_to_message_id != top.id:
        # massege is a reply
        msgs_quotes.setdefault(d.reply_to_message_id, 0)
        msgs_quotes[d.reply_to_message_id] += 1

    if d.text:
        tops[top.id][0] += 1
        tops[top.id][1] += len(d.text)
        tops[top.id][2].append(d)


New top: 4621
New top: 2789
New top: 2488
New top: 2340
New top: 92406
New top: 5489
New top: 7265
New top: 27200
New top: 319436
New top: 4639
New top: 7626
New top: 2374
New top: 25827
New top: 19091
New top: 2776
New top: 105418
New top: 3389
New top: 26730
New top: 2623
New top: 326314


In [36]:
print(msgs[27200])

{
    "_": "Message",
    "id": 27200,
    "from_user": {
        "_": "User",
        "id": 1989050175,
        "is_self": false,
        "is_contact": false,
        "is_mutual_contact": false,
        "is_deleted": false,
        "is_bot": false,
        "is_verified": false,
        "is_restricted": false,
        "is_scam": false,
        "is_fake": false,
        "is_support": false,
        "is_premium": true,
        "first_name": "Vladyslav",
        "last_name": "Mazurak",
        "status": "UserStatus.RECENTLY",
        "username": "Vladyslav_Mazurak",
        "emoji_status": {
            "_": "EmojiStatus",
            "custom_emoji_id": 5990240103761449952
        },
        "dc_id": 4,
        "photo": {
            "_": "ChatPhoto",
            "small_file_id": "AQADBAADOb8xG1ZyAVIAEAIAAz9_jnYABJBHn0nIrnEcAAQeBA",
            "small_photo_unique_id": "AgADOb8xG1ZyAVI",
            "big_file_id": "AQADBAADOb8xG1ZyAVIAEAMAAz9_jnYABJBHn0nIrnEcAAQeBA",
            "big_phot

In [15]:
t = [(k, v[0], v[1]) for k, v in sorted(tops.items(), key=lambda x: -x[1][0])]
t

[(105418, 151, 12636),
 (2340, 109, 8788),
 (4621, 64, 4031),
 (92406, 44, 2527),
 (7265, 33, 1846),
 (5489, 21, 1160),
 (26730, 16, 2465),
 (2488, 12, 941),
 (7626, 7, 314),
 (27200, 5, 293),
 (19091, 4, 350),
 (2776, 4, 680),
 (3389, 4, 498),
 (319436, 3, 555),
 (2623, 3, 102),
 (2789, 2, 219),
 (4639, 1, 35),
 (2374, 1, 1030),
 (326314, 1, 40),
 (25827, 0, 0)]

In [28]:
'''
[(92406, [32431, 2169905]), # Srachivnya
 (4621, [5423, 361251]),    # Itshechna
 (3389, [4291, 371518]),    # Pravo
 (5489, [3920, 307077]),    # Avto
 (105418, [3785, 329030]),  # Relocation
 (26730, [2938, 231552]),   # Consoli
 (7626, [2694, 194551]),    # Finances
 (2340, [2215, 178146]),    # Common Chat
 (2776, [1594, 162379]),    # Housing
 (4705, [655, 43829]),      # Culture
 (2488, [610, 61006]),      # Travel
 (25827, [569, 55870]),     # Job Search
 (2623, [533, 47695]),      # Medical
 (4639, [509, 33378]),      # Food
 (1149, [280, 21072]),      # Mental Health
 (7265, [181, 14294]),      # SecondHand
 (19091, [160, 21207]),     # Kids
 (4638, [150, 14610]),      # Pets
 (319436, [145, 14163]),    # Investments
 (27200, [143, 8621]),      # Memes
'''

def get_user(m):
    return m.from_user.username or f"{m.from_user.first_name}({m.from_user.id})"


thread_id = 92406
data = []
start = -100
end = None
for i, m in enumerate(tops[thread_id][2][start:end]):
    if m.from_user and m.text:
        msg = [f"== {m.date} ==\n"]
        bases = []
        base = m
        while base.reply_to_message_id != thread_id:
            base = msgs.get(base.reply_to_message_id)
            if base and base.text and base.from_user:
                base_user = get_user(base)
                for l in reversed(base.text.splitlines()):
                    bases.append(f"{base_user} {base.id} > {l}")
        if bases:
            bases = "\n".join(reversed(bases)) + "\n"
            msg.append(bases)
        user = get_user(m)
        msg.append(f"{user} {m.id}> {m.text}\n")
        if m.reactions and m.reactions.reactions:
            for r in m.reactions.reactions:
                msg.append(f"{r.emoji} {r.count} ")
            msg.append("\n")
        if m.id in msgs_quotes:
            msg.append(f"replies: {msgs_quotes[m.id]}\n")            
        msg.append("\n")
        data.append("".join(msg))
    if i == 0:
        print(m.date)
print(m.date)


2024-09-21 21:02:12
2024-09-22 18:42:27


In [30]:
print(data[4])

== 2024-09-21 21:46:48 ==
Ooooooles 325651 > Ребзі, мам питаніє, мені знайомий казав шо десь чув шо з наст року планують прибрать б2б якщо в тебе 1 контракт. Хтось чув за таке, може вусатий там шось розказував чи ще хтось, бо я кароч не чув
pescaao 325777 > Ну знайдемо друзів з грузинськими фопами, або бельгійськими, або ще з якимось. І будемо робити один одному фактури по three hundred bucks 😊
yuuzoo 325780 > А що за прикол з бельгійськими фопами?
pescaao 325781> В них є вимога ця на більше двох контрактів. І вони не в Польщі




In [31]:
l = len("".join(data))
print(l, l/2.72)

13147 4833.4558823529405


In [26]:
import openai
key=open("key.txt").read().strip()
client = openai.OpenAI(api_key=key)

In [33]:
data_txt = "".join(data)

sys_prompt = """
Task: Summarize Conversations from Telegram Channel

1. Generate a summary of the discussions within the channel.
2. Identify the top 10 specific conversations, do not use general terms like "taxes",
    "adaptation" and so on, use specific terms from chat.
    For each conversation:
      * List the main participants involved.
      * Provide a detailed summary in the original language.
      * include 5 most important quotes (include ({message_id} {username}) in quote)
      * Final conclusion
3. Tone Analysis: Note that many comments may carry a sarcastic tone. If it’s ambiguous whether a statement is sincere or not, interpret it as having an ironic or mocking tone.
4. Zrada vs. Peremoga (ZvP) Rating: Assign a ZVP rating for each topic on a scale from -5 (complete betrayal) to 5 (complete victory). Justify the assigned value with clear reasoning.
5. Overall Summary: Conclude with an overall sentiment analysis of the chat and its cumulative ZvP rating.
6. Participant Activity: Compile a list of the top participants along with:
  * A summary of their activity.
  * 3 top quotes
  * An individual ZvP rating for each participant, with explanations.
    
Common terms:
 ZUS - Poland goverment agency for social insurance
 PIT - income tax
 "ухилянт" - person who left Ukraine to avoid being mobilised
 many other terms are based on Polish to Ukrainian phonetic adaptation

People related context:
 * Most of participants of this chat are ukrainians living in Poland and working in software development related companies.
 * Some of them left Ukraine before war, but most people moved to Poland due to russian invasion.
 * Some people already moved from Poland to other countries, some people are planning this.

Notes:
 * Provide a comprehensive response to ensure all elements are thoroughly covered and avoid prematurely ending the message.
 * **Generate answer in Ukrainian language.**
 * Use informal playful tone.
 * Keep chat-specific slang terms as is.
 * Do not use term "sarcastic", "joke", "joking" too much, use appropriate replacement terms.
     Do not focus on this "joking" aspect to much, find some other points.
 * Focus on recent messages.
 * Generate a concise list of topics first as "content index" and final response after that.

Example output:
====
**Топ обговорення:**

1. Обговорення відео з Instagram
2. Дискусія про фінансові вимоги жінок
3. Обговорення кави та кав'ярень
4. Заклик до мобілізації українців за кордоном
5. Обговорення демографічної ситуації в Україні
6. Обговорення мусульман в Кракові та Закопанах)

---

**1. Обговорення відео з Instagram**

Основні учасники: hvala_bok, Artem, IllyaVolkov, andrii, is_lll, werq142

Стислий опис: Учасники обговорюють відео з Instagram, в якому жінка висловлює свої вимоги до потенційного партнера. Дискусія зосереджена на її прямолінійності та вимогах.

Цитати:
- "Яка прекрасна людина" (249976 Artem)
- "Ну якщо не рофл то і так молодець - чесно заявила свою позицію" (249982 is_lll)
- "Просто треба більше заробляти" (250000 hvala_bok)
- "А для чого тобі тоді таке чудо?" (249989 Oleksandr)
- "Ну відразу сказала що не для слабонєрвних" (249986 werq142)

Висновок: Учасники жартують про вимоги жінки, обговорюючи, чи є вони обґрунтованими.

ZvP: -1 (легка зрада) - обговорення з іронічним тоном, що підкреслює нереалістичність вимог.

---

**2. Дискусія про фінансові вимоги жінок**

Основні учасники: mykytasahan7, jack_bauer_d, Oleksandr, Andrii99887

Стислий опис: Учасники обговорюють фінансові вимоги жінок, які очікують, що партнери покриватимуть їхні витрати. Дискусія переходить до обговорення ролі жінок у стосунках.

Цитати:
- "А що вона мені може дати і нащо вона така взагалі" (250064 Andrii99887)
- "Якшо вона тіпу зайде прям пздц - буде горіти нею, то він має підпадати під її умови" (250065 jack_bauer_d)
- "Просто треба більше заробляти" (250000 hvala_bok)
- "Ну різні ситуації є" (250055 Oleksandr)
- "Це як ракова пухлина, яка передається через Тік ток і інсті" (250054 Andrii99887)

Висновок: Учасники висловлюють скептицизм щодо таких вимог, підкреслюючи, що не всі жінки мають такі очікування.

ZvP: -2 (зрада) - обговорення з негативним тоном щодо вимог жінок.

---

**3. Обговорення кави та кав'ярень**

Основні учасники: Artem, werq142, Konark96, Andrii99887

Стислий опис: Учасники обговорюють ціни на каву в Польщі, різні види кави та кавомашини. Дискусія зосереджена на якості кави та вартості.

Цитати:
- "Кавова шиза у польщі присутня у явній формі" (250183 Artem)
- "Часто капучино з автоматичної каво машини краще ніж те що роблять ці срані баристи" (250208 Andrii99887)
- "У мене є кавʼярня у домі, кава 16 плн" (250178 Artem)
- "В жабці по 6-8" (250192 werq142)
- "Лавацу взагалі не треба брати, є польскі паларні кави дуже добрі" (250227 Artem)

Висновок: Учасники обговорюють переваги кавомашин перед кав'ярнями, підкреслюючи високу вартість кави в Польщі.

ZvP: 0 (нейтрально) - обговорення без особливого емоційного забарвлення.

---

**4. Заклик до мобілізації українців за кордоном**

Основні учасники: hvala_bok, Konark96, m_di_di, jack_bauer_d

Стислий опис: Обговорення заклику до мобілізації українців, які живуть за кордоном. Учасники висловлюють скептицизм та обговорюють можливі наслідки.

Цитати:
- "Готуємся посони, за сцикунів скоро візьмуться" (250318 hvala_bok)
- "Так він же сказав що зброї у рф більше в декілька разів, сенс тоді?" (250321 Konark96)
- "То ми вже до Аргентини приєднуємося" (250330 m_di_di)
- "Сирський закликав українців, які живуть за кордоном, взяти участь у війні" (250382 hvala_bok)
- "Ну і ясно що це не буде анкета в легіон на паспорт бо вже всі собі зробили" (250388 hvala_bok)

Висновок: Учасники висловлюють іронічний скептицизм щодо мобілізації, підкреслюючи, що багато хто вже має плани на еміграцію.

ZvP: -3 (значна зрада) - обговорення з негативним тоном щодо мобілізації.

---

**5. Обговорення демографічної ситуації в Україні**

Основні учасники: Andrii99887, zaz1104

Стислий опис: Учасники обговорюють демографічну ситуацію в Україні, жартуючи про необхідність народжувати більше дітей.

Цитати:
- "Не народила 3 дітей не жінка" (250468 Andrii99887)
- "Мінімум 12" (250471 zaz1104)
- "Чоловіки вмирають, жінки народжують" (250478 Andrii99887)
- "Ідеальний світ" (250478 Andrii99887)
- "Тоді і 12 стрільне" (250477 zaz1104)

Висновок: Обговорення з іронічним тоном, підкреслюючи нереалістичність таких очікувань.

ZvP: -1 (легка зрада) - обговорення з іронічним тоном.

---

**6. Обговорення мусульман в Кракові та Закопанах**

Основні учасники: igorzharii, olekoleko

Стислий опис: Учасники обговорюють велику кількість мусульман в Кракові та Закопанах, жартуючи про це.

Цитати:
- "Чуєте а хто з Кракова" (250503 igorzharii)
- "А чого у вас стільки мусульман/арабів" (250505 igorzharii)
- "Тут якась мусульманська мекка як для євреїв в Умані?" (250507 igorzharii)
- "З білорусі набігли" (250511 olekoleko)

Висновок: Обговорення з іронічним тоном, підкреслюючи несподіваність такої ситуації.

ZvP: 0 (нейтрально) - обговорення без особливого емоційного забарвлення.

---

**Загальний підсумок:**

Загальний тон чату іронічний та скептичний, з акцентом на обговорення нереалістичних вимог та ситуацій. Учасники часто жартують і висловлюють скептицизм щодо різних тем, від мобілізації до фінансових вимог жінок.

ZvP: -2 (зрада) - загальний негативний тон обговорень, з акцентом на скептицизм та іронію.

---

**Активність учасників:**

1. **Artem(667857019)**
   - Активність: Часто коментує різні теми, особливо обговорення кави та фінансових вимог.
   - Топ цитати:
     - "Кавова шиза у польщі присутня у явній формі" (250183)
     - "У мене є кавʼярня у домі, кава 16 плн" (250178)
     - "Яка прекрасна людина" (249976)
   - ZvP: -1 (легка зрада) - часто іронізує та висловлює скептицизм.

2. **hvala_bok**
   - Активність: Ініціює обговорення мобілізації та відео з Instagram.
   - Топ цитати:
     - "Готуємся посони, за сцикунів скоро візьмуться" (250318)
     - "Просто треба більше заробляти" (250000)
     - "Сирський закликав українців, які живуть за кордоном, взяти участь у війні" (250382)
   - ZvP: -2 (значна зрада) - висловлює скептицизм щодо мобілізації.

3. **Andrii99887**
   - Активність: Активно обговорює фінансові вимоги жінок та демографічну ситуацію.
   - Топ цитати:
     - "А що вона мені може дати і нащо вона така взагалі" (250064)
     - "Часто капучино з автоматичної каво машини краще ніж те що роблять ці срані баристи" (250208)
     - "Не народила 3 дітей не жінка" (250468)
   - ZvP: -2 (значна зрада) - висловлює скептицизм та іронію щодо різних тем.

4. **Konark96**
   - Активність: Обговорює мобілізацію та каву.
   - Топ цитати:
     - "Так він же сказав що зброї у рф більше в декілька разів, сенс тоді?" (250321)
     - "В регіонах капучино 10-12" (250178)
     - "Ну я і у Варшаві знайду за 10" (250199)
   - ZvP: -1 (легка зрада) - висловлює скептицизм щодо мобілізації та цін на каву.

5. **mykytasahan7**
   - Активність: Обговорює фінансові вимоги жінок та мобілізацію.
   - Топ цитати:
     - "да там пиздець" (250382)
     - "в рашці 150 млн" (250392)
     - "А що вона мені може дати і нащо вона така взагалі" (250064)
   - ZvP: -2 (значна зрада) - висловлює скептицизм та іронію щодо різних тем.
===
"""

prompt = f"""This is a history of telegram chat:
===========
{data_txt}
===========
"""

# Call the API for chat completions
response = client.chat.completions.create(
    model="gpt-4o-2024-08-06",  # or use the model you prefer
    #model="gpt-4o-mini",  # or use the model you prefer
    messages=[{"role": "system", "content": sys_prompt},{"role": "user", "content": prompt}],
    temperature=0.3,
    max_completion_tokens=5000,
)


# Get the summary from the response
summary = response.choices[0].message.content
print(summary)

**Топ обговорення:**

1. Обговорення можливого скасування B2B контрактів
2. Дискусія про інженерів на бурових установках
3. Обговорення української мережі в Польщі
4. Дискусія про фізичну активність та кількість кроків

---

**1. Обговорення можливого скасування B2B контрактів**

Основні учасники: Ooooooles, pescaao, yuuzoo, alexzhv13

Стислий опис: Учасники обговорюють чутки про можливе скасування B2B контрактів з одним клієнтом у Польщі. Вони розглядають різні способи обходу цієї ситуації, такі як використання грузинських або бельгійських ФОПів.

Цитати:
- "Ну знайдемо друзів з грузинськими фопами, або бельгійськими, або ще з якимось." (325777 pescaao)
- "В описі будемо писати 'fisting', а в Польщі це взагалі не оподатковується" (325779 pescaao)
- "А що за прикол з бельгійськими фопами?" (325780 yuuzoo)
- "Там же податки за 45% перевалить" (325782 yuuzoo)
- "Спердалать :)" (325784 yuuzoo)

Висновок: Учасники іронічно обговорюють можливі обхідні шляхи для збереження B2B контрактів, пі

In [34]:
print(response.usage)

CompletionUsage(completion_tokens=2177, prompt_tokens=8258, total_tokens=10435, completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0))
